# Chapter 11 Guide

## 11.4.1

In [ ]:
import os  #Aimport requests  #BSERPAPI_KEY = os.getenv("SERPAPI_KEY")  #Cdef search_product_urls(search_key: str, num_results: int = 5) -> list[dict]:  #D    """Search for product page candidates using SerpAPI."""    params = {  #E        "q": search_key,        "api_key": SERPAPI_KEY,        "num": num_results,        "engine": "google",    }    resp = requests.get("https://serpapi.com/search", params=params)  #F    resp.raise_for_status()    data = resp.json()        candidates = []  #G    for result in data.get("organic_results", []):        candidates.append({            "title": result.get("title", ""),            "url": result.get("link", ""),            "snippet": result.get("snippet", ""),            "position": result.get("position", 0),        })    return candidates  #H# Example usagecandidates = search_product_urls("GORUCK GR1 26L")  #Ifor c in candidates:    print(f"  [{c['position']}] {c['title']}")    print(f"       {c['url']}")

## 11.4.2

In [ ]:
import openai  #Afrom pydantic import BaseModel  #Bclass URLRanking(BaseModel):  #C    best_url: str    confidence: str  # "high", "medium", "low"    reasoning: strdef rank_urls_with_ai(  #D    search_key: str,    candidates: list[dict],    model: str = "gpt-4o",) -> URLRanking:    """Use an LLM to pick the best product page from search results."""        candidate_text = ""  #E    for c in candidates:        candidate_text += (            f"Position {c['position']}:\n"            f"  Title: {c['title']}\n"            f"  URL: {c['url']}\n"            f"  Snippet: {c['snippet']}\n\n"        )        system_prompt = """You are a data engineering assistant helping build a product database.Given a product search key and a list of candidate URLs from search results,pick the single best URL for extracting structured product data.Prefer:1. Manufacturer or official brand pages2. Pages likely to contain: product name, price, description, weight, images3. Individual product pages over category or listing pages4. Stable URLs over session-specific or filtered URLsAvoid:- Review sites, forums, Reddit threads- Retailer pages when a manufacturer page is available- Category pages that list multiple productsReturn the best URL, your confidence level, and a brief explanation."""  #F        user_prompt = (        f"Product: {search_key}\n\nCandidate URLs:\n{candidate_text}"    )  #G        response = openai.beta.chat.completions.parse(  #H        model=model,        messages=[            {"role": "system", "content": system_prompt},            {"role": "user", "content": user_prompt},        ],        response_format=URLRanking,  #I    )    return response.choices[0].message.parsed  #J# Example usageranking = rank_urls_with_ai("GORUCK GR1 26L", candidates)  #Kprint(f"Best URL: {ranking.best_url}")print(f"Confidence: {ranking.confidence}")print(f"Reasoning: {ranking.reasoning}")

## 11.5.1

In [ ]:
from bs4 import BeautifulSoup  #AREMOVE_TAGS = [  #B    "script", "style", "nav", "footer", "header",    "iframe", "noscript", "svg", "form",]REMOVE_CLASSES = [  #C    "breadcrumb", "related-products", "recently-viewed",    "newsletter", "cookie-banner", "site-footer",    "site-header", "cart-drawer", "search-modal",    "review", "reviews", "ratings",]def clean_html_aggressive(html: str) -> str:  #D    """Remove non-product HTML elements to reduce noise and token count."""    soup = BeautifulSoup(html, "html.parser")        # Remove unwanted tags entirely    for tag_name in REMOVE_TAGS:  #E        for element in soup.find_all(tag_name):            element.decompose()        # Remove elements by class name patterns    for class_pattern in REMOVE_CLASSES:  #F        for element in soup.find_all(            class_=lambda c: c and class_pattern in " ".join(c).lower()        ):            element.decompose()        # Remove empty elements    for element in soup.find_all():  #G        if not element.get_text(strip=True) and not element.find("img"):            element.decompose()        clean_text = " ".join(soup.stripped_strings)  #H    return clean_text# Example usageraw_html = requests.get("https://www.goruck.com/products/gr1").text  #Iclean = clean_html_aggressive(raw_html)print(f"Raw HTML: {len(raw_html):,} characters")print(f"Cleaned:  {len(clean):,} characters")print(f"Reduction: {(1 - len(clean) / len(raw_html)) * 100:.0f}%")

## 11.5.2

In [ ]:
from pydantic import BaseModel  #Aclass ContentTriage(BaseModel):  #B    product_sections: list[str]    non_product_sections: list[str]def triage_content(text_blocks: list[str], model: str = "gpt-4o-mini") -> ContentTriage:  #C    """Use a lightweight LLM to classify text blocks as product or non-product."""    blocks_text = ""  #D    for i, block in enumerate(text_blocks):        blocks_text += f"[Block {i}]: {block[:200]}\n\n"        system_prompt = """You are a data engineering assistant. Given a list of text blocksfrom a product web page, classify each block as either product-relevant or not.Product-relevant blocks contain: product name, price, description, specifications,weight, dimensions, materials, features, or sizing information.Non-product blocks contain: navigation, shipping info, return policies, reviews,promotional banners, newsletter signups, or generic site content.Return two lists: product_sections (the block numbers that contain product data)and non_product_sections (everything else)."""  #E        response = openai.beta.chat.completions.parse(  #F        model=model,        messages=[            {"role": "system", "content": system_prompt},            {"role": "user", "content": blocks_text},        ],        response_format=ContentTriage,    )    return response.choices[0].message.parsed  #G

## 11.6.1

In [ ]:
from pydantic import BaseModel, Field  #Afrom typing import Optional  #Bclass ProductExtraction(BaseModel):  #C    """Schema for extracting product data from web page content."""    product_name: str = Field(  #D        description="Full product name as shown on the page"    )    brand_name: str = Field(  #E        description="Manufacturer or brand name"    )    description: Optional[str] = Field(  #F        default=None,        description="Product description, typically 1-3 sentences"    )    price: Optional[str] = Field(  #G        default=None,        description="Current retail price including currency symbol"    )    weight: Optional[str] = Field(  #H        default=None,        description="Product weight with unit (e.g., '2.5 lbs', '1.1 kg')"    )    primary_image_url: Optional[str] = Field(  #I        default=None,        description="URL of the main product image"    )    category: Optional[str] = Field(  #J        default=None,        description="Product category (e.g., backpack, tent, sleeping bag)"    )

## 11.6.2

In [ ]:
import openai  #AEXTRACTION_PROMPT = """You are a product data extraction assistant for a data engineering pipeline.Given the text content of a product web page, extract the following fields accurately:- product_name: The full product name as displayed on the page- brand_name: The manufacturer or brand- description: A concise product description (1-3 sentences)- price: The current retail price with currency symbol- weight: The product weight with unit if available- primary_image_url: The URL of the main product image if found in the text- category: The product category (backpack, tent, sleeping bag, headlamp, etc.)Rules:- Only extract information that is explicitly present in the text- Use null for any field you cannot find or confidently determine- Do not guess or fabricate values- For price, use the current or sale price, not the original price if both are shown- For weight, include the unit (lbs, oz, kg, g)- For category, use a simple label based on what the product is"""  #Bdef extract_product_with_ai(  #C    cleaned_text: str,    model: str = "gpt-4o",) -> ProductExtraction:    """Extract product fields from cleaned page text using an LLM."""    response = openai.beta.chat.completions.parse(  #D        model=model,        messages=[            {"role": "system", "content": EXTRACTION_PROMPT},            {"role": "user", "content": cleaned_text},  #E        ],        response_format=ProductExtraction,  #F    )    return response.choices[0].message.parsed  #G# Example usageproduct = extract_product_with_ai(clean)  #Hprint(f"Name:        {product.product_name}")print(f"Brand:       {product.brand_name}")print(f"Price:       {product.price}")print(f"Weight:      {product.weight}")print(f"Category:    {product.category}")print(f"Description: {product.description[:100] if product.description else None}...")

## 11.6.3

In [ ]:
import pandas as pd  #Afrom bs4 import BeautifulSoup  #Bdef extract_manual_goruck(html: str) -> dict:  #C    """Manual extraction using Chapter 10's CSS selector approach."""    soup = BeautifulSoup(html, "html.parser")        title_el = soup.find("h1")    title = title_el.get_text(" ", strip=True) if title_el else None        price_el = (        soup.select_one(            "div.product-block__price span.price-item--sale.price-item--last"        )        or soup.select_one("div.product-block__price span.price-item--regular")    )    price = price_el.get_text(" ", strip=True) if price_el else None        return {        "product_name": title,        "brand_name": "GORUCK",  # hardcoded for this site        "price": price,        "weight": None,  # manual approach did not extract this        "category": None,  # manual approach did not extract this        "description": None,  # manual approach did not extract this    }# Run both approaches on the same pageurl = "https://www.goruck.com/products/gr1"  #Draw_html = requests.get(url).textmanual_result = extract_manual_goruck(raw_html)  #Ecleaned = clean_html_aggressive(raw_html)ai_result = extract_product_with_ai(cleaned)  #F# Build comparison tablecomparison = pd.DataFrame({  #G    "Field": ["product_name", "brand_name", "price", "weight",              "category", "description"],    "Manual": [        manual_result["product_name"],        manual_result["brand_name"],        manual_result["price"],        manual_result["weight"],        manual_result["category"],        manual_result["description"],    ],    "AI": [        ai_result.product_name,        ai_result.brand_name,        ai_result.price,        ai_result.weight,        ai_result.category,        ai_result.description[:80] if ai_result.description else None,    ],})print(comparison.to_string(index=False))

## 11.7.1

In [ ]:
import time  #Aimport pandas as pd  #Btest_products = [  #D    {"name": "GORUCK GR1 26L",     "url": "https://www.goruck.com/products/gr1"},    {"name": "Osprey Atmos AG 65",     "url": "https://www.osprey.com/us/en/product/atmos-ag-65-ATMOS65S23.html"},    {"name": "Petzl Actik Core",     "url": "https://www.petzl.com/US/en/Sport/PERFORMANCE-headlamps/ACTIK-CORE"},]results = []  #Efor product in test_products:    record = {  #F        "search_key": product["name"],        "url": product["url"],        "status": "error",    }    try:        raw_html = requests.get(product["url"]).text  #G        cleaned = clean_html_aggressive(raw_html)        extraction = extract_product_with_ai(cleaned)  #H                record["product_name"] = extraction.product_name        record["brand_name"] = extraction.brand_name        record["price"] = extraction.price        record["weight"] = extraction.weight        record["category"] = extraction.category        record["description"] = (            extraction.description[:80] + "..."            if extraction.description            else None        )        record["status"] = "success"  #I    except Exception as e:        record["status"] = f"error: {type(e).__name__}"  #J        results.append(record)    time.sleep(1)  #Kout = pd.DataFrame(results)  #Lprint(out[["search_key", "status", "product_name", "price", "weight", "category"]]      .to_string(index=False))

## 11.8

In [ ]:
import tiktoken  #Adef estimate_extraction_cost(  #B    text: str,    model: str = "gpt-4o",    output_tokens: int = 300,) -> dict:    """Estimate the token count and cost for an extraction call."""    encoder = tiktoken.encoding_for_model(model)  #C    input_tokens = len(encoder.encode(text))  #D        # Pricing as of mid-2025 (check docs for current rates)    pricing = {  #E        "gpt-4o": {"input": 2.50 / 1_000_000, "output": 10.00 / 1_000_000},        "gpt-4o-mini": {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},    }        rates = pricing.get(model, pricing["gpt-4o"])  #F    input_cost = input_tokens * rates["input"]    output_cost = output_tokens * rates["output"]    total_cost = input_cost + output_cost        return {  #G        "input_tokens": input_tokens,        "output_tokens_est": output_tokens,        "input_cost": round(input_cost, 6),        "output_cost": round(output_cost, 6),        "total_cost": round(total_cost, 6),    }# Estimate for a single product pagecost = estimate_extraction_cost(clean)  #Hprint(f"Input tokens:  {cost['input_tokens']:,}")print(f"Output tokens: {cost['output_tokens_est']:,} (estimated)")print(f"Cost per page: ${cost['total_cost']:.4f}")print(f"Cost for 450 products: ${cost['total_cost'] * 450:.2f}")